In [53]:
from pylab import *
import matplotlib.pyplot as plt
import matplotlib as mpimg
from scipy.misc import imresize
import matplotlib
import pdb
import numpy as np
import cv2
import moviepy
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import os, sys

%matplotlib inline

In [54]:
def region_of_interest(img):
    
    
    mask = np.zeros(img.shape, dtype=np.uint8) #mask image
    roi_corners = np.array([[(0,500), (960,500), (540,330),(440,330)]], 
                           dtype=np.int32) # vertisies seted to form trapezoidal scene
    channel_count = img.shape[2]  # image channels
    ignore_mask_color = (255,)*channel_count
    cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    

    # apply the mask
    masked_image = cv2.bitwise_and(img, mask)
 
    return masked_image

In [55]:
def prepareImage (rI,f):
   
    hlsI=cv2.cvtColor(rI, cv2.COLOR_BGR2HLS) # convert to HLS to anlaysis the lane agains the amount of light
    
#     hlsE = np.zeros((540,960,3), 'uint8') 
    hlsI[:,:,0]=0                            # hlsE image with setting Hue channel to zero
   
    
    
    bgrI=cv2.cvtColor(hlsI, cv2.COLOR_HLS2BGR)    # convert to BGR

    blur = blur_image(bgrI)                       #Filter image
 
    return  blur

In [56]:
def blur_image (image):
    return cv2.bilateralFilter(image,7,150,150) # this filter chosen due to it's performance in sharping edges 


In [57]:
def cannyEdges(masked, low_threshold, high_threshold):
    return cv2.Canny(masked, low_threshold, high_threshold)

In [58]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, rPrev,lPrev,rPrevS,lPrevS):
   
      
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
   
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    lL=[] #Left lines
    rL=[] #Right lines
    
    
    
    for line in lines: # classify right and left lines and ignore  horizontal lines or any line with high slope
        for x1,y1,x2,y2 in line:
            slope  = (y2 - y1) / (x2 - x1)
                        
            if (slope>.5):
                lL.append(line)
            elif (slope<-.5):
                rL.append(line)
    if not lL:                    #if left line = null , draw previuos line with red color
        color=[255,0,0]    
        draw_lines(line_img, lPrev,lPrevS,color)
        
    else:                        # if left line , average all lines by averging the poits x1,y1,x2,y2
        
        lL=np.mean(lL,axis=0) 
        lL=lL.astype(int32)
        slope  = (lL[0][3] - lL[0][1]) / (lL[0][2] - lL[0][0])
        draw_lines(line_img, lL[0],slope) # draw line 
        lPrev=lL[0] # make current prevline= current line
        lPrevS=slope
        lL=[]
        
        
        
    if not rL:                   #if right line = null , draw previuos line with red color
        color=[255,0,0]        
        draw_lines(line_img, rPrev,rPrevS,color)
        
    else:                         # if right line , average all lines by averging the poits x1,y1,x2,y2
        rL=np.mean(rL,axis=0) 
        rL=rL.astype(int32)
        slope  = (rL[0][3] - rL[0][1]) / (rL[0][2] - rL[0][0])
        
          
        draw_lines(line_img, rL[0],slope)    # Draw line
        rPrev=rL[0]    # make prev right line = current right line
        rPrevS=slope
        rL=[]
        

    
    
    
    
    return line_img, rPrev,lPrev, rPrevS, lPrevS

In [59]:
def draw_lines(img, lines,s, color=[0, 250, 0], thickness=5):
 
    
    x1=lines[0]
    y1=lines[1]
    x2=lines[2]
    y2=lines[3]
    
    #lenAB = sqrt(pow(x1 - x2, 2.0) + pow(y1 - y2, 2.0)); # Calculate length of line
    
    if (s>.5):
        if (x1<x2):       # Aligning the line with fixed y1, y2
            x1=x2
            y1,y2=y2,y1
            
                       
        
        y2=330
        x2=int((y2-y1+s*x1)/s)
        y1=500
        x1=int((y1-y2+s*x2)/s)
        
    elif (s<-.5):
        if (x1>x2):      # Aligning the line with fixed y1,y2
            
            x2=x1
            y1,y2=y2,y1
           
      
        y2=330
        x2=int((y2-y1+s*x1)/s)
        y1=500
        x1=int((y1-y2+s*x2)/s)
        
   
    cv2.line(img, (x1, y1), (x2, y2), color, thickness,lineType=cv2.LINE_AA) # Draw line
    
   
      
   

In [60]:
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):  
  
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [61]:
from moviepy.editor import * 
from IPython import display
from IPython.core.display import display
from IPython.display import Image
import pylab
from scipy.misc import imresize

import time

t=0
clip = VideoFileClip('challenge.mp4') # Read clip
#fourcc = cv2.VideoWriter_fourcc('M','J','P','G')      #OpenCv writing file
#out = cv2.VideoWriter('Detected_Lane3.avi',fourcc, 15.0, (960, 540)) # intialise output file
rPrevS=0.0
lPrevS=0.0
rPrev=[] # Previuos right line
lPrev=[] # Previuos left line


d=0      # Frame number
%time
rI=[]      #frames array
for frames in clip.iter_frames():

        
    frames = imresize(frames, (540,960)) # resize frame to fit all frames size(all test videos) 


    ROI=region_of_interest(frames) #extract ROI

    masked_data = prepareImage(ROI,d)  # enhance the Image   

    edges= cannyEdges(masked_data,200,255) # edge detection

    [ll, rPrev, lPrev, rPrevS, lPrevS]=hough_lines(edges, 1, np.pi/180, 20, 30, 200, rPrev,lPrev,rPrevS,lPrevS)  # ll=Lines image mask
    result= weighted_img(ll, frames, α=0.8, β=1., λ=0.) #output frame

    rI.append(result)

   # out.write(result)  #write output

    d=d+1

#out.release()
%time
    
    # etc.

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs


In [62]:
from moviepy.editor import *

clip_out="Detected Lane Output.mp4"
new_clip = ImageSequenceClip(rI, fps=15)
new_clip.write_videofile(clip_out)



[MoviePy] >>>> Building video Detected Lane Output.mp4
[MoviePy] Writing video Detected Lane Output.mp4


100%|██████████| 251/251 [00:03<00:00, 78.54it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Detected Lane Output.mp4 



In [63]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(clip_out))

